In [7]:
from pathlib import Path
import os
from glob import glob
import subprocess
import shutil

In [8]:
def get_x_y_z(path):
    with open(path, 'r') as file:
        for line in file:
            if line.strip().endswith("APPROX POSITION XYZ"):
                return float(line.split()[0]), float(line.split()[1]), float(line.split()[2])
            

def convert_to_enu(year, path_result, path_rinex, station):
    try:
        dir_for_year = os.listdir(f"./{year}/")[:-1]
    except:
        dir_for_year = -1
        logging.error(f"no data for the year {year}")
    if dir_for_year != -1:
        x, y, z = get_x_y_z(min(glob(f"{path_rinex}/{year}/{station}/*o"), key=os.path.getctime))
        for day_dir in dir_for_year:
            files_to_process = glob(f"./{year}/{day_dir}/kin*")
            Path(f"{path_result}_ENU/{year}/{day_dir}").mkdir(parents=True, exist_ok=True)
            Path(f"{path_result}_XYZ/{year}/{day_dir}").mkdir(parents=True, exist_ok=True)
            for f in files_to_process:
                out_name = f[-12:]
                cp = subprocess.run(["wsl",  "bash", "-ic", f"xyz2enu ./{year}/{day_dir}/kin_{out_name} {path_result}_ENU/{year}/{day_dir}/enu_{out_name} {x} {y} {z}"], capture_output=True)
                shutil.move(f"./{year}/{day_dir}/kin_{out_name}", f"{path_result}_XYZ/{year}/{day_dir}/kin_{out_name}")
                if cp.returncode!=0:
                    print(cp)
                else:
                    print(f"{f} processed successfully")

In [9]:
convert_to_enu(2017, "./results_test2", "./raw_rinex", 'albh')

In [46]:
def clean_rep(year, path_rinex):
    try:
        shutil.rmtree(f"{path_rinex}{year}")
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))

In [49]:
path_rinex = './raw_rinex/'
year = 2018
clean_rep(year, path_rinex)